In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import catboost
import nltk

In [3]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
test_labels = pd.read_csv("./data/test_labels.csv")

## Подробнее о метках (labels)
Все они принимают значение -1, 0, 1
* Insult - Оскорбление
* Obscene - Непристойный
* Severe toxicity - Сильная токсичность
* Toxicity - Токсичность
* Threat - Угроза
* Identity hate - Ненависть к идентичности

In [4]:
# sample_submission.head()

In [5]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [8]:
# train.loc[(train['toxic'] > 0) & ((train['severe_toxic'] == 0) & (train['obscene'] == 0) & (train['threat'] == 0) & (train['insult'] == 0) & (train['identity_hate'] == 0))]

То, что в ячейке выше строки не пустые нам говорит о том, что toxic необязательно содержит в себе остальные метки.

In [9]:
# train.loc[(train['toxic'] == 0) & ((train['severe_toxic'] != 0) | (train['obscene'] != 0) | (train['threat'] != 0) | (train['insult'] != 0) | (train['identity_hate'] != 0))]


А это говорит о том, что оскорбления, брань и прочее могут присутствовать даже в нетоксичных комментариях

In [10]:
cols = "toxic severe_toxic obscene threat insult identity_hate"
for i in cols.split():
    print(train[i].value_counts())

0    144277
1     15294
Name: toxic, dtype: int64
0    157976
1      1595
Name: severe_toxic, dtype: int64
0    151122
1      8449
Name: obscene, dtype: int64
0    159093
1       478
Name: threat, dtype: int64
0    151694
1      7877
Name: insult, dtype: int64
0    158166
1      1405
Name: identity_hate, dtype: int64


Выше просто статистика по данным, можем оценить, какие значения у нас бывают

In [11]:
# Тут мы создаём серию отфильтрованных комментариев, чтобы попробовать работать с ними, а не с сырым материалом

import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

stop_words.update("wikipedia article page talk".split())
def preprocess_text(sen):
    # Удаление символов пунктуации и специальных символов

    sentence = re.sub('[^a-zA-Z0-9]', ' ', sen)

    # Заменяем любые пробелы на один обычный
    sentence = re.sub(r'\s+', ' ', sentence)
    # Делаем всё в нижнем регистре
    sentence = sentence.lower()
    # Удаляем стоп-слова
    sentence = " ".join([word for word in sentence.split() if word not in stop_words])
    return sentence

filtered_train = train['comment_text'].apply(preprocess_text)
filtered_train

0         explanation edits made username hardcore metal...
1         aww matches background colour seemingly stuck ...
2         hey man really trying edit war guy constantly ...
3         make real suggestions improvement wondered sec...
4                                  sir hero chance remember
                                ...                        
159566    second time asking view completely contradicts...
159567              ashamed horrible thing put 128 61 19 93
159568    spitzer umm theres actual prostitution ring cr...
159569    looks like actually put speedy first version d...
159570    really think understand came idea bad right aw...
Name: comment_text, Length: 159571, dtype: object

In [12]:
from collections import Counter
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

# all_words = " ".join(filtered_train).split()
# word_counts = Counter(all_words)
# top_words = dict(word_counts.most_common(100))
# wordcloud = WordCloud(background_color="white")
# wordcloud.generate_from_frequencies(top_words)
# plt.figure(figsize=(10,10))
# plt.axis('off')
# plt.imshow(wordcloud)

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(train['comment_text'])

In [25]:
tfidf_matrix
sc


MemoryError: Unable to allocate 225. GiB for an array with shape (159571, 189630) and data type float64

In [17]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
